In [ ]:
from pyspark import *;
from pyspark.sql import *;
from pyspark.sql.functions import *;
import pandas as pd;

In [ ]:
spark = SparkSession.builder.appName("Q4").getOrCreate()

In [ ]:
df = spark.read.csv("audible_uncleaned.csv",header=True,inferSchema=True)
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- author: string (nullable = true)
 |-- narrator: string (nullable = true)
 |-- time: string (nullable = true)
 |-- releasedate: string (nullable = true)
 |-- language: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- price: string (nullable = true)

+--------------------+--------------------+--------------------+------------------+-----------+--------+--------------------+--------+
|                name|              author|            narrator|              time|releasedate|language|               stars|   price|
+--------------------+--------------------+--------------------+------------------+-----------+--------+--------------------+--------+
|Geronimo Stilton ...|Writtenby:Geronim...|Narratedby:BillLo...| 2 hrs and 20 mins| 04-08-2008| English|5 out of 5 stars3...|     468|
|    The Burning Maze|Writtenby:RickRio...|Narratedby:Robbie...| 13 hrs and 8 mins| 01-05-2018| English|4.5 out of 5 star...|     820|
|        The

In [ ]:
df.select([count(when(isnull(c)|isnan(c),c)).alias(c) for c in df.columns]).show()

+----+------+--------+----+-----------+--------+-----+-----+
|name|author|narrator|time|releasedate|language|stars|price|
+----+------+--------+----+-----------+--------+-----+-----+
|   0|    11|       0|   0|          0|       0|   14|    0|
+----+------+--------+----+-----------+--------+-----+-----+



In [ ]:
df = df.withColumn("price",regexp_replace("price","[$,]",""))
df = df.withColumn("price",col("price").cast("int"))
df.show()

+--------------------+--------------------+--------------------+------------------+-----------+--------+--------------------+-----+
|                name|              author|            narrator|              time|releasedate|language|               stars|price|
+--------------------+--------------------+--------------------+------------------+-----------+--------+--------------------+-----+
|Geronimo Stilton ...|Writtenby:Geronim...|Narratedby:BillLo...| 2 hrs and 20 mins| 04-08-2008| English|5 out of 5 stars3...|  468|
|    The Burning Maze|Writtenby:RickRio...|Narratedby:Robbie...| 13 hrs and 8 mins| 01-05-2018| English|4.5 out of 5 star...|  820|
|        The Deep End|Writtenby:JeffKinney|Narratedby:DanRus...|  2 hrs and 3 mins| 06-11-2020| English|4.5 out of 5 star...|  410|
|Daughter of the Deep|Writtenby:RickRio...|Narratedby:Soneel...|11 hrs and 16 mins| 05-10-2021| English|4.5 out of 5 star...|  615|
|The Lightning Thi...|Writtenby:RickRio...|Narratedby:JesseB...|            

In [ ]:
df = df.fillna({"author":"Writtenby:Unknown"})
df = df.dropna(subset=["stars"])

In [ ]:
df = df.withColumn("author",regexp_replace("author","Writtenby:",""))
df = df.withColumn("narrator",regexp_replace("narrator","Narratedby:",""))
df.show()

+--------------------+----------------+--------------------+------------------+-----------+--------+--------------------+-----+
|                name|          author|            narrator|              time|releasedate|language|               stars|price|
+--------------------+----------------+--------------------+------------------+-----------+--------+--------------------+-----+
|Geronimo Stilton ...| GeronimoStilton|          BillLobely| 2 hrs and 20 mins| 04-08-2008| English|5 out of 5 stars3...|  468|
|    The Burning Maze|     RickRiordan|       RobbieDaymond| 13 hrs and 8 mins| 01-05-2018| English|4.5 out of 5 star...|  820|
|        The Deep End|      JeffKinney|          DanRussell|  2 hrs and 3 mins| 06-11-2020| English|4.5 out of 5 star...|  410|
|Daughter of the Deep|     RickRiordan|      SoneelaNankani|11 hrs and 16 mins| 05-10-2021| English|4.5 out of 5 star...|  615|
|The Lightning Thi...|     RickRiordan|      JesseBernstein|            10 hrs| 13-01-2010| English|4.5 

In [ ]:
df.select(min("price")).show()
df.select(max("price")).show()
df = df.filter((col("price")>=100)& (col("price")<=2000))

+----------+
|min(price)|
+----------+
|        32|
+----------+

+----------+
|max(price)|
+----------+
|    263566|
+----------+



In [ ]:
df.show()

+--------------------+----------------+--------------------+------------------+-----------+--------+--------------------+-----+
|                name|          author|            narrator|              time|releasedate|language|               stars|price|
+--------------------+----------------+--------------------+------------------+-----------+--------+--------------------+-----+
|Geronimo Stilton ...| GeronimoStilton|          BillLobely| 2 hrs and 20 mins| 04-08-2008| English|5 out of 5 stars3...|  468|
|    The Burning Maze|     RickRiordan|       RobbieDaymond| 13 hrs and 8 mins| 01-05-2018| English|4.5 out of 5 star...|  820|
|        The Deep End|      JeffKinney|          DanRussell|  2 hrs and 3 mins| 06-11-2020| English|4.5 out of 5 star...|  410|
|Daughter of the Deep|     RickRiordan|      SoneelaNankani|11 hrs and 16 mins| 05-10-2021| English|4.5 out of 5 star...|  615|
|The Lightning Thi...|     RickRiordan|      JesseBernstein|            10 hrs| 13-01-2010| English|4.5 

In [ ]:
hours = regexp_extract(col("time"), r"(\d+)\s*hrs", 1)
minutes = regexp_extract(col("time"), r"(\d+)\s*mins", 1)

df = df.withColumn("hours", when(hours == "", 0).otherwise(hours.cast("int")))
df = df.withColumn("minutes", when(minutes == "", 0).otherwise(minutes.cast("int")))
df = df.withColumn("total_time_mins", (col("hours") * 60 + col("minutes")))

df = df.drop("hours","minutes")
df.show()

+--------------------+----------------+--------------------+------------------+-----------+--------+--------------------+-----+---------------+
|                name|          author|            narrator|              time|releasedate|language|               stars|price|total_time_mins|
+--------------------+----------------+--------------------+------------------+-----------+--------+--------------------+-----+---------------+
|Geronimo Stilton ...| GeronimoStilton|          BillLobely| 2 hrs and 20 mins| 04-08-2008| English|5 out of 5 stars3...|  468|            140|
|    The Burning Maze|     RickRiordan|       RobbieDaymond| 13 hrs and 8 mins| 01-05-2018| English|4.5 out of 5 star...|  820|            788|
|        The Deep End|      JeffKinney|          DanRussell|  2 hrs and 3 mins| 06-11-2020| English|4.5 out of 5 star...|  410|            123|
|Daughter of the Deep|     RickRiordan|      SoneelaNankani|11 hrs and 16 mins| 05-10-2021| English|4.5 out of 5 star...|  615|         

In [ ]:
df = df.withColumn("releasedate", to_date(col("releasedate"), "dd-MM-yyyy"))
df.show()

+--------------------+----------------+--------------------+------------------+-----------+--------+--------------------+-----+---------------+
|                name|          author|            narrator|              time|releasedate|language|               stars|price|total_time_mins|
+--------------------+----------------+--------------------+------------------+-----------+--------+--------------------+-----+---------------+
|Geronimo Stilton ...| GeronimoStilton|          BillLobely| 2 hrs and 20 mins| 2008-08-04| English|5 out of 5 stars3...|  468|            140|
|    The Burning Maze|     RickRiordan|       RobbieDaymond| 13 hrs and 8 mins| 2018-05-01| English|4.5 out of 5 star...|  820|            788|
|        The Deep End|      JeffKinney|          DanRussell|  2 hrs and 3 mins| 2020-11-06| English|4.5 out of 5 star...|  410|            123|
|Daughter of the Deep|     RickRiordan|      SoneelaNankani|11 hrs and 16 mins| 2021-10-05| English|4.5 out of 5 star...|  615|         

In [ ]:
df = df.withColumn("stars1", regexp_extract(col("stars"), r"(\d+(\.\d+)?)", 1).cast("double"))
df = df.withColumn("ratings", regexp_extract(col("stars"), r"stars(\d+)", 1).cast("int"))
df.show()

+--------------------+----------------+--------------------+------------------+-----------+--------+--------------------+-----+---------------+------+-------+
|                name|          author|            narrator|              time|releasedate|language|               stars|price|total_time_mins|stars1|ratings|
+--------------------+----------------+--------------------+------------------+-----------+--------+--------------------+-----+---------------+------+-------+
|Geronimo Stilton ...| GeronimoStilton|          BillLobely| 2 hrs and 20 mins| 2008-08-04| English|5 out of 5 stars3...|  468|            140|   5.0|     34|
|    The Burning Maze|     RickRiordan|       RobbieDaymond| 13 hrs and 8 mins| 2018-05-01| English|4.5 out of 5 star...|  820|            788|   4.5|     41|
|        The Deep End|      JeffKinney|          DanRussell|  2 hrs and 3 mins| 2020-11-06| English|4.5 out of 5 star...|  410|            123|   4.5|     38|
|Daughter of the Deep|     RickRiordan|      S

In [ ]:
df = df.toPandas()
df.to_csv("cleaned_audible.csv")